In [1]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [4]:
#getting the already scraped dict
import json
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/vdict VI-FR ver 3.json') as json_file:
     currDict = json.load(json_file)

In [5]:
#reading the word list produced by an crubadan:
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/anCruVietList') as infile:
  wordsData = infile.readlines()

In [6]:
#generate a list of words from the word list produced by an crubadan
def generateWordList(): 
  temp = []
  for word in wordsData:
    parts = word.strip().split(" ")
    #ignore the frequency
    temp.append(" ".join(parts[1:]))
  return temp

In [7]:
wordList = generateWordList()
len(wordList)

34275

In [8]:
wordList[0:5]

['việt', 'việt nam', 'có thể', 'lý', 'i']

In [9]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [10]:
for batch in batches:
  print(len(batch))

10000
10000
10000
4275


In [11]:
print(len(batches))

4


In [12]:
#The link format for vdict Viet-Fran is: https://vdict.com/m%E1%BB%99t,4,0,0.html

In [13]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [14]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",4,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"

    translations = {}

    #Getting the first translation
    try:
      curTag = soup.find("ul", class_= "list1")
      if (curTag.find('b') is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"
    except:
      print("Empty translation page " + word)
      return "N/A"

    #storing the number of translations a word has
    numTrans = 1

    #continue while we still have more translations
    while(curTag != '\n'): 
        temp = {} #temp dict to store each translation of a word

        #getting the raw translation
        if (curTag.find('b').string is not None): #empty translation cell
            temp['translation'] = re.split('[,;]', curTag.find('b').string)

            #retrieving the examples of each of the translations
            try:
                examples = curTag.findAll("ul", class_= "list2")
                temp2 = {} #another dict to store each examples of a translation

                numExamples = len(list(examples))
                temp2['nums'] = numExamples

                for j in range(0, numExamples):
                    temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                    #retrieving the usage of each examples
                    try: 
                        temp2['usage ' + str(j+1)] = str(examples[j].find("li").contents[-1])
                    except:
                        temp2['usage ' + str(j+1)] = "N/A"

                temp['examples'] = temp2
            except:
                temp['examples'] = "N/A"

            translations[numTrans] = temp #adding the translation to the dictionary

            numTrans += 1
        #go to next tag  
        curTag = curTag.next_sibling
    
    translations["nums"] = numTrans - 1
        
    return translations

In [15]:
url = "https://vdict.com/" + quote("có") + ",4,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')


#make sure the word is in the dictionary
if (soup.find("div", id = 'result-contents') is None):
    print("Not in the dictionary")

In [16]:
soup.find("ul", class_= "list1").next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [17]:
containers = soup.findAll("ul", class_= "list1")
containers

[<ul class="list1"><li><b>avoir; posséder</b><ul class="list2"><li><span class="example-original">Tôi có nói câu đó </span><br/>j'ai dit cette phrase</li></ul><ul class="list2"><li><span class="example-original">Có tiền </span><br/>avoir de l'argent</li></ul><ul class="list2"><li><span class="example-original">Có một cái nhà </span><br/>posséder une maison</li></ul></li></ul>,
 <ul class="list1"><li><b>exister; se trouver</b><ul class="list2"><li><span class="example-original">Cái tục đó còn đó </span><br/>cet usage existe encore</li></ul><ul class="list2"><li><span class="example-original">Loài chim đó không có ở Việt Nam </span><br/>cette espèce d'oiseau n'existe pas au Viet-Nam</li></ul><ul class="list2"><li><span class="example-original">Tên anh không có trong danh sách </span><br/>ton nom ne se trouve pas sur la liste</li></ul></li></ul>,
 <ul class="list1"><li><b>disposer de</b><ul class="list2"><li><span class="example-original">ông ấy có một cái xe </span><br/>il dispose d'une 

In [18]:
meanings = [item.find('b').string for item in containers[0:4]] #word translation
meanings

['avoir; posséder', 'exister; se trouver', 'disposer de', 'être présent']

In [19]:
examples = containers[0].findAll("ul", class_= "list2")
examples[1].find("li").contents[-1]

"avoir de l'argent"

In [20]:
from google.colab import files
start_time = time.time()

index = 4
batch = batches[index] 

#for (index, batch) in enumerate(batches):
dictionary = {}

#making the dictionary for each batch
for word in batch:

    #if the word is not in already in the dict, find the translation
    if word not in currDict:
      dictionary[word] = findTranslation(word)
      time.sleep(1) #wait for 1 sec between each query

IndexError: ignored

In [ ]:
#saving the batch
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/vdict VI-FR ver 4 batch {}.json'.format(index), 'w') as outfile:
  json.dump(dictionary, outfile)
  
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(dictionary.items())

In [ ]:
findTranslation("đo lường")